# Detailed Chemistry Modeling in PROMMIS: Tutorial for Chemical Precipitation Applications
*A step-by-step guide for modeling aqueous reactions, pH control, and mineral scaling using integrated Reaktoro and PROMMIS models*


## 1. Introduction

Chemical precipitation is a core mechanism in water treatment, hydrometallurgy, brine management, and critical mineral recovery. It describes the process in which dissolved ions combine to form a solid mineral phase once their activities exceed the solubility limit of that mineral.

In this tutorial, we cover a general workflow for:

- Modeling precipitation of various **critical minerals** (e.g., carbonate, sulfate, hydroxide solids)
- Adding reagents to control **pH**, **alkalinity**, and **ionic strength**
- Predicting mineral **scaling** during brine concentration
- Setting up and solving a reduced-order **PROMMIS** property model




## 2. What Is Chemical Precipitation?

Chemical precipitation occurs when the **ionic activity product (IAP)** of certain ions exceeds a mineral’s **solubility product (Ksp)**:

$$
\text{IAP} = \prod_i a_i^{\nu_i},\qquad
\text{If IAP} > K_\text{sp},\ \text{mineral formation occurs.}
$$

Typical mineral categories for critical mineral and brine systems include:

- **Carbonates** (e.g., calcite, dolomite, siderite)  
- **Sulfates** (e.g., gypsum, barite)  
- **Hydroxides** (e.g., Mg(OH)₂, Fe(OH)₃)  
- **Mixed solid solutions** (e.g., transition-metal hydroxides or carbonates)

Reaktoro allows defining any number of aqueous species and solids from selected thermodynamic databases (e.g., PHREEQC, SUPCRT, EQ3/6).



## 3. Precipitation Processes Example Case

In this example, the **untreated wastewater** would be treated by adding **lime** ($\text{CaO}$) to raise the $\text{pH}$ greater than 7, and **calcite** ($\text{CaCO}_3$) would be the precipitant in the sludge.

---

### Inlet Water Composition

The inlet water composition is given below (concentrations in $\text{mg/L}$ as the ion):

| Ion | Concentration ($\text{mg/L}$) |
| :--- | :--- |
| $\text{Na}^+$ | 4 |
| $\text{K}^+$ | 4.8 |
| $\text{Ca}^{2+}$ | 1052 |
| $\text{Mg}^{2+}$ | 435 |
| $\text{Cl}^-$ | 174.1 |
| $\text{SO}_4^{2-}$ | 3508 |
| $\text{HCO}_3^-$ | 940.8 |

---

### Reagent Addition

Add reagents **Sodium Carbonate** ($\text{Na}_2\text{CO}_3$) and **Calcium Oxide (Lime)** ($\text{CaO}$) as $\text{pH}$ adjusters, with doses of **\*\*\* $\text{mg/L}$** and **\*\*\* $\text{mg/L}$**, respectively.

<img src="pfd.png" alt="Chemical Precipitation PFD" width="80%">

# 4. Model Development for Precipitation Processes with Detailed Chemistry

Model Development for Precipitation Processes with Detailed Chemistry and Firstly Develop Detail Chemistry Model 

---

### 4.1. Import Modules from the Library

In [2]:
from pyomo.environ import (
    Var,
    Param,
    Constraint,
    Expression,
    Objective,
    ConcreteModel,
    Block,
    value,
    assert_optimal_termination,
    units as pyunits,
)

# Ideas core components
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import (
    set_scaling_factor,
    constraint_scaling_transform,
)
from idaes.core.util.model_statistics import degrees_of_freedom

from pyomo.util.calc_var_value import calculate_variable_from_constraint

# Import reaktoro-pse and reaktoro
from reaktoro_pse.reaktoro_block import ReaktoroBlock
import reaktoro
from reaktoro_pse.core.util_classes.cyipopt_solver import (
    get_cyipopt_watertap_solver,
)

### 4.2. Create Process Flowsheet Model to Begin the Model Development


In [3]:
m = ConcreteModel()
# create IDAES flowsheet
m.fs = FlowsheetBlock(dynamic=False)

## 4.3. Create the Feed Brine Composition and Convert to Reaktoro-Enabled Format

In [4]:
# Brine input assumptions in example case
brine_feed_composition = {
    "Na": 4,
    "K": 4.8,
    "Ca": 1052,
    "Mg": 435,
    "Cl": 174.1,
    "SO4": 3508,
    "HCO3": 940.8,
}
ion_mw = {
    "Na": 22.989769 / 1000,
    "K": 39.0983 / 1000,
    "Ca": 40.078 / 1000,
    "Mg": 24.305 / 1000,
    "Cl": 35.453 / 1000,
    "SO4": 96.06 / 1000,
    "HCO3": 61.0168 / 1000,
    "H2O": 18.01528 / 1000,
}
brine_feed_ph = 7.6

m.fs.feed = Block()
ions = list(brine_feed_composition.keys())
m.fs.feed.species_concentrations = Var(
    ions, initialize=1, bounds=(0, None), units=pyunits.mg / pyunits.L
)
ions.append("H2O")
m.fs.feed.species_mass_flow = Var(
    ions, initialize=1, bounds=(0, None), units=pyunits.kg / pyunits.s
)
m.fs.feed.specie_mw = Param(ions, units=pyunits.kg / pyunits.mol)
m.fs.feed.pH = Var(initialize=brine_feed_ph)
m.fs.feed.temperature = Var(initialize=293, units=pyunits.K)
m.fs.feed.pressure = Var(initialize=1e5, units=pyunits.Pa)
m.fs.feed.density = Var(initialize=1053, units=pyunits.kg / pyunits.m**3)

m.fs.feed.pH.fix()
m.fs.feed.temperature.fix()
m.fs.feed.pressure.fix()
m.fs.feed.density.fix()


# convert concentration to mass flows as Reaktoro-Enabled Format
@m.fs.feed.Constraint(list(m.fs.feed.species_mass_flow.keys()))
def eq_feed_species_mass_flow(fs, ion):
    if ion == "H2O":
        return Constraint.Skip
    else:
        return m.fs.feed.species_mass_flow[ion] == pyunits.convert(
            m.fs.feed.species_concentrations[ion]
            * m.fs.feed.species_mass_flow["H2O"]
            / m.fs.feed.density,
            to_units=pyunits.kg / pyunits.s,
        )

## 4.4. Add Precipitation Assumptions and Additional Constraints for Reaktoro Block Calculation

In [5]:
m.fs.softening = Block()
m.fs.softening.reagents_mol_flow = Var(
    ["CaO", "Na2CO3"],
    initialize=1e-7,
    bounds=(1e-10, None),
    units=pyunits.mol / pyunits.s,
)

m.fs.softening.reagent_dose = Var(
    ["CaO", "Na2CO3"], initialize=1e-7, units=pyunits.mg / pyunits.kg
)
m.fs.softening.reagents_mw = Param(["CaO", "Na2CO3"], units=pyunits.kg / pyunits.mol)
m.fs.softening.reagents_mw["CaO"] = 56.0774 / 1000
m.fs.softening.reagents_mw["Na2CO3"] = 105.99 / 1000

m.fs.softening.precipitants = Var(["Calcite"], initialize=1e-32, units=pyunits.mol)
m.fs.softening.precipitants_mw = Param(
    ["Calcite"], initialize=100.09 / 1000, units=pyunits.kg / pyunits.mol
)

m.fs.softening.effluent_pH = Var(
    initialize=7, bounds=(5, 12), units=pyunits.dimensionless
)
m.fs.softening.effluent_alkalinity = Var(
    initialize=100, units=pyunits.mg / pyunits.L, bounds=(30, None)
)
m.fs.softening.effluent_species_mass_flows = Var(
    ions, initialize=1e-5, units=pyunits.kg / pyunits.s
)


@m.fs.softening.Constraint(ions)
def eq_effluent_species_mass_flows(fs, ion):
    if "Ca" == ion:
        return m.fs.softening.effluent_species_mass_flows["Ca"] == (
            m.fs.softening.reagents_mol_flow["CaO"] * m.fs.feed.specie_mw["Ca"]
            + m.fs.feed.species_mass_flow["Ca"]
            - m.fs.softening.precipitants["Calcite"] * m.fs.feed.specie_mw["Ca"]
        )
    elif "Na" == ion:
        return m.fs.softening.effluent_species_mass_flows["Na"] == (
            2 * m.fs.softening.reagents_mol_flow["Na2CO3"] * m.fs.feed.specie_mw["Na"]
            + m.fs.feed.species_mass_flow["Na"]
        )
    elif "HCO3" == ion:
        return m.fs.softening.effluent_species_mass_flows["HCO3"] == (
            m.fs.feed.species_mass_flow["HCO3"]
            + m.fs.softening.reagents_mol_flow["Na2CO3"] * m.fs.feed.specie_mw["HCO3"]
            - m.fs.softening.precipitants["Calcite"] * m.fs.feed.specie_mw["HCO3"]
        )
    else:
        return (
            m.fs.softening.effluent_species_mass_flows[ion]
            == m.fs.feed.species_mass_flow[ion]
        )


@m.fs.softening.Constraint(["CaO", "Na2CO3"])
def eq_dose(fs, reagent):
    return m.fs.softening.reagent_dose[reagent] == pyunits.convert(
        m.fs.softening.reagents_mol_flow[reagent]
        * m.fs.softening.reagents_mw[reagent]
        / m.fs.feed.species_mass_flow["H2O"],
        to_units=pyunits.mg / pyunits.kg,
    )

## 4.5. Added Reaktoro Blocks into the Process for Detailed Chemistry Calculation

In [6]:
m.fs.softening.reaktoro_outputs = {
    (
        "speciesAmount",
        "Calcite",
    ): m.fs.softening.precipitants["Calcite"],
    ("alkalinityAsCaCO3", None): m.fs.softening.effluent_alkalinity,
    ("pH", None): m.fs.softening.effluent_pH,
}
m.fs.softening.eq_precipitation = ReaktoroBlock(
    aqueous_phase={
        "composition": m.fs.feed.species_mass_flow,
        "convert_to_rkt_species": True,
        "activity_model": reaktoro.ActivityModelPitzer(),
        "fixed_solvent_specie": "H2O",
    },
    system_state={
        "temperature": m.fs.feed.temperature,
        "pressure": m.fs.feed.pressure,
        "pH": m.fs.feed.pH,
    },
    mineral_phase={"phase_components": ["Calcite"]},
    chemistry_modifier={
        "CaO": m.fs.softening.reagents_mol_flow["CaO"],
        "Na2CO3": m.fs.softening.reagents_mol_flow["Na2CO3"],
    },
    outputs=m.fs.softening.reaktoro_outputs,
    database_file="pitzer.dat",
    build_speciation_block=True,
)

2025-12-09 14:15:45 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding H
2025-12-09 14:15:45 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding O
2025-12-09 14:15:45 [INFO] idaes.reaktoro_pse.core.reaktoro_solver: rktSolver inputs: ['[Cl]', '[C]', '[Na]', '[Mg]', '[S]', '[K]', '[Ca]', '[H2O]', '[H]', '[O]', '[H+]']
2025-12-09 14:15:45 [INFO] idaes.reaktoro_pse.core.reaktoro_solver: rktSolver constraints: ['charge', 'C_constraint', 'Na_constraint', 'Mg_constraint', 'S_constraint', 'K_constraint', 'Ca_constraint', 'H2O_constraint', 'H_dummy_constraint', 'O_dummy_constraint', 'pH']
2025-12-09 14:15:45 [INFO] idaes.reaktoro_pse.core.reaktoro_solver: rktSolver inputs: ['[H+]', '[H]', '[C]', '[O]', '[Na]', '[Mg]', '[S]', '[Cl]', '[K]', '[Ca]']
2025-12-09 14:15:45 [INFO] idaes.reaktoro_pse.core.reaktoro_solver: rktSolver constraints: ['charge', 'H_constrain

## 4.6. Setting Scaling Factors, Initializing the Process, and Solving the Detailed Chemistry Estimation

In [7]:
m.fs.feed.species_mass_flow["H2O"].fix(1.0)  # Fixing water flow to 1 kg/s
# fix concentrations
for ion, val in brine_feed_composition.items():
    m.fs.feed.species_concentrations[ion].fix(val)
    m.fs.feed.specie_mw[ion] = ion_mw[ion]
    set_scaling_factor(m.fs.feed.species_concentrations[ion], 1 / val)
m.fs.feed.specie_mw["H2O"] = ion_mw["H2O"]
for comp, pyoobj in m.fs.feed.eq_feed_species_mass_flow.items():
    calculate_variable_from_constraint(m.fs.feed.species_mass_flow[comp], pyoobj)
    set_scaling_factor(
        m.fs.feed.species_mass_flow[ion], 1 / m.fs.feed.species_mass_flow[comp].value
    )
    constraint_scaling_transform(pyoobj, 1 / m.fs.feed.species_mass_flow[comp].value)
set_scaling_factor(m.fs.feed.density, 1 / 1000)
set_scaling_factor(m.fs.feed.pH, 1)
set_scaling_factor(m.fs.feed.temperature, 1 / 273)
set_scaling_factor(m.fs.feed.pressure, 1e-5)
m.fs.softening.reagents_mol_flow["CaO"].fix(1e-5)
m.fs.softening.reagents_mol_flow["Na2CO3"].fix(1e-10)  # 1e-5


for comp, pyoobj in m.fs.softening.eq_effluent_species_mass_flows.items():
    calculate_variable_from_constraint(
        m.fs.softening.effluent_species_mass_flows[comp], pyoobj
    )
    # Scale based on feed
    set_scaling_factor(
        m.fs.softening.effluent_species_mass_flows[ion],
        1 / m.fs.feed.species_mass_flow[comp].value,
    )
    constraint_scaling_transform(pyoobj, 1 / m.fs.feed.species_mass_flow[comp].value)

for reagent in m.fs.softening.reagents_mol_flow.keys():
    set_scaling_factor(m.fs.softening.reagents_mol_flow[reagent], 1e4)
    set_scaling_factor(m.fs.softening.reagent_dose[reagent], 1)
    constraint_scaling_transform(m.fs.softening.eq_dose[reagent], 1)
for reagent in m.fs.softening.precipitants.keys():
    set_scaling_factor(m.fs.softening.precipitants[reagent], 1e4)
set_scaling_factor(m.fs.softening.effluent_pH, 1)
m.fs.softening.eq_precipitation.initialize()

for key, obj in m.fs.softening.reaktoro_outputs.items():
    print(key, value(obj))

# recompute effluent composition using updated amount of formed calcite.
for comp, pyoobj in m.fs.softening.eq_effluent_species_mass_flows.items():
    calculate_variable_from_constraint(
        m.fs.softening.effluent_species_mass_flows[comp], pyoobj
    )

print(degrees_of_freedom(m))
assert degrees_of_freedom(m) == 0
solver = get_cyipopt_watertap_solver(max_iter=200, linear_solver="mumps", pivtol=1e-4)

result = solver.solve(m, tee=True)
assert_optimal_termination(result)

2025-12-09 14:15:56 [INFO] idaes.reaktoro_pse.reaktoro_block: ---initializing property block fs.softening.eq_precipitation----
2025-12-09 14:15:56 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully
2025-12-09 14:15:56 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully
2025-12-09 14:15:57 [WARNING] idaes.reaktoro_pse.core.reaktoro_block_builder: Jacobian scale for ('alkalinityAsCaCO3', None) below 1e-08, set to 1e-08
2025-12-09 14:15:57 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Initialized rkt block
('speciesAmount', 'Calcite') 0.0033595219273473416
('alkalinityAsCaCO3', None) 741.8626142494027
('pH', None) 6.55004785622937
0
cyipopt-watertap: cyipopt with user variable scaling and IDAES jacobian constraint scaling

List of user-set options:

                                    Name   Value                used
              acceptable_constr_viol_tol = 1e-09                 yes
                 acceptable_dual_inf_tol = 0.01      

## 4.7. Display the Results

In [8]:
m.fs.softening.reagent_dose["CaO"].value
m.fs.softening.reagent_dose["Na2CO3"].value
m.fs.softening.effluent_pH.value
print("Quicklime Dose")
print(m.fs.softening.reagent_dose["CaO"].value)
print(" pH")
print(m.fs.softening.effluent_pH.value)
print("calcium_removal")
print(
    100
    * (
        1
        - m.fs.softening.effluent_species_mass_flows["Ca"].value
        / (
            m.fs.feed.species_mass_flow["Ca"].value
            + m.fs.softening.reagents_mol_flow["CaO"].value
            * m.fs.softening.reagents_mw["CaO"].value
        )
    )
)
print("carbonate_removal")
print(
    100
    * (
        (
            m.fs.feed.species_mass_flow["HCO3"].value
            - m.fs.softening.effluent_species_mass_flows["HCO3"].value
        )
        / m.fs.feed.species_mass_flow["HCO3"].value
    )
)

Quicklime Dose
0.560774
 pH
6.550047856177478
calcium_removal
13.485535820242845
carbonate_removal
22.943409527784627


## 4.8. Estimate the Parameters for Model Integration with Prommis


## 4.9. Create the Prommis Process Model and Property Models with Example Case Assumptions and Preload Default Parameters Before Model Integration

In [ ]:
from customized_liquid_properties import AqueousParameter
from customized_liquid_properties import PrecipitateParameters
from prommis.precipitate.precipitator import Precipitator


m.fs.properties_aq = AqueousParameter()
m.fs.properties_solid = PrecipitateParameters()

m.fs.unit = Precipitator(
    property_package_aqueous=m.fs.properties_aq,
    property_package_precipitate=m.fs.properties_solid,
)

m.fs.unit.aqueous_inlet.flow_vol[0].fix(10000)

    "Na": 4,
    "K": 4.8,
    "Ca": 1052,
    "Mg": 435,
    "Cl": 174.1,
    "SO4": 3508,
    "HCO3": 940.8,

m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Na"].fix(4)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "K"].fix(4.8)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Ca"].fix(1052)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Mg"].fix(435)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "Cl"].fix(174.1)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "SO4"].fix(3508)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "HCO3"].fix(940.8)
m.fs.unit.aqueous_inlet.conc_mass_comp[0, "H2O"].fix(1000000)

m.fs.unit.cv_precipitate[0].temperature.fix(298.15)

assert_units_consistent(m.fs.unit)
assert degrees_of_freedom(m) == 0
initializer = BlockTriangularizationInitializer(constraint_tolerance=2e-5)
initializer.initialize(m.fs.unit)
solver = get_solver()
results = solver.solve(m)
assert_optimal_termination(results)

## 4.10. Integrate the Reaktoro Model and Prommis Model by Populating the Estimated Detailed Chemistry Parameters into the Property Model and Re-solve the Process Model

## 4.11. Display the Integrated Prommis Model Results

